In [ ]:
from unityagents import UnityEnvironment
import numpy as np
import random

from collections import deque
import matplotlib.pyplot as plt
from ddpg_agent import Agent
from model import Actor, Critic

import torch

%matplotlib inline


In [ ]:
env = UnityEnvironment(file_name='../../Reacher_v1.app')

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [ ]:
# reset the environment
env_info = env.reset(train_mode=False)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

In [ ]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

random_seed=0
#Agent.actor_local = Actor(state_size, action_size, random_seed).to(device)
#Agent.actor_local.load_state_dict(torch.load('checkpoint_actor_v1.pth'))

agent = Agent(state_size=state_size, action_size=action_size, random_seed=10)
# Agent.actor_local = Actor(state_size, action_size, random_seed).to(device)
agent.actor_local.load_state_dict(torch.load('checkpoint_actor_v1.pth'))

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
agent.reset()

while True:
    #actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    #actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    action = agent.act(state)
    
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:
env.reset()